In [3]:
import sys
!{sys.executable} -m pip install sklearn

  Using cached sklearn-0.0.tar.gz (1.1 kB)
  Created wheel for sklearn: filename=sklearn-0.0-py2.py3-none-any.whl size=1310 sha256=0cac82414c4e4a7c3a1b0ece85c109c3fa1d582c1ce442fde1e8a6b8d05bd8d7
  Stored in directory: /Users/aristophanesalbertusalvin/Library/Caches/pip/wheels/e4/7b/98/b6466d71b8d738a0c547008b9eb39bf8676d1ff6ca4b22af1c
Successfully built sklearn


In [18]:
from ast import Mult
from sklearn.datasets import load_files
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer

from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier

In [6]:
# Loading the data and making the train-test splits.

review_files = load_files('./cleaned_op_spam', encoding='latin-1')
x_train, x_test, y_train, y_test = train_test_split(review_files.data, review_files.target, test_size=0.2)
# print(review_files)

In [7]:
# Since we're working with text data, we need to vectorize the documents / turn each document
# into a point vector in a n-dimentions space. 
# Moreover, rather than simply using the word counts, we want to use the Term Frequency as feature value.

vectorizer = TfidfVectorizer(stop_words='english')
x_trainvec = vectorizer.fit_transform(x_train)
x_testvec = vectorizer.transform(x_test)
# print(x_trainvec.toarray())
# print(vectorizer.get_feature_names_out())

In [8]:
mnb = MultinomialNB()
mnb.fit(x_trainvec, y_train)
y_preds = mnb.predict(x_testvec)
# print('Length predictions', len(y_preds))
# print('Length y_test', len(y_test))

In [7]:
correct_pred = 0
for i in range(len(y_test)):
    if y_preds[i] == y_test[i]:
        correct_pred += 1

print(f"Correct predictions {correct_pred} out of {len(y_test)}")

Correct predictions 262 out of 320


In [25]:
def run_cross_validation(nfolds, ngram=1):
    tf_vectorizer = TfidfVectorizer(stop_words='english', ngram_range=(ngram, ngram))
    x_train_vec = tf_vectorizer.fit_transform(review_files.data)
    y_train_vec = review_files.target

    mnbayes = MultinomialNB()
    logreg = LogisticRegression()
    dtree = DecisionTreeClassifier()
    ranForest = RandomForestClassifier(n_estimators=10, max_depth=None, min_samples_split=2, random_state=0)
    
    ac_bayes = cross_val_score(mnbayes, x_train_vec, y_train_vec, cv=nfolds, scoring='accuracy').mean()
    ac_logreg = cross_val_score(logreg, x_train_vec, y_train_vec, cv=nfolds, scoring='accuracy').mean()
    ac_dtree = cross_val_score(dtree, x_train_vec, y_train_vec, cv=nfolds, scoring='accuracy').mean()
    ac_ranForest = cross_val_score(ranForest, x_train_vec, y_train_vec, cv=nfolds, scoring='accuracy').mean()
    
    print('Naive Bayes:', ac_bayes)
    print(f'Logistic Regression: { ac_logreg }')
    print(f'Decision Tree: { ac_dtree }')
    print(f'Random Forest: { ac_ranForest }')

In [26]:
print('===== BASIC ACCURACIES =====')
run_cross_validation(nfolds=5, ngram=1)
print('\n===== BIGRAM ACCURACIES =====')
run_cross_validation(nfolds=5, ngram=2)

===== BASIC ACCURACIES =====
Naive Bayes: 0.79125
Logistic Regression: 0.8174999999999999
Decision Tree: 0.5075000000000001
Random Forest: 0.5931249999999999

===== BIGRAM ACCURACIES =====
Naive Bayes: 0.630625
Logistic Regression: 0.7181250000000001
Decision Tree: 0.49375
Random Forest: 0.5075
